In [1]:
import numpy as np
from scipy.stats import binom
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import interact, interact_manual, widgets,fixed
import functools

In [20]:
def generate_biased_rnd_bin(p_in,n_in):
    v_raw_rnd_bin=np.array(np.random.rand(n_in)<p_in,dtype=np.uint8)
    return v_raw_rnd_bin

def test_T1(v_rnd,Th_low,Th_high):
    T1=v_rnd.sum()
    verdict_pass=(T1>Th_low) & (T1<Th_high)
    return verdict_pass

screen_result=widgets.Text(value="",description='', disabled=True,layout=widgets.Layout(width='25%'),font_size=12)
screen_entropy=widgets.Text(value="",description='', disabled=True,layout=widgets.Layout(width='25%'),font_size=12)

def run_several_monobits(p=0.5,nb_tests=100,test_size=512,Th_low=250,Th_high=260,screen_result_in=screen_result,screen_entropy_in=screen_entropy):
    cnt_fail=0
    for i in range(nb_tests):
        v_rnd=generate_biased_rnd_bin(p,test_size)
        cnt_fail+=not(test_T1(v_rnd,Th_low,Th_high))
    s_line_dur='Le test a leve une alarme  %d fois sur 100'%(cnt_fail,nb_tests)
    screen_result_in.value=s_line_dur
    if p*(1-p)!=0:
        entropy=((p)*np.log2(p)+(1-p)*np.log2(1-p))*-1
    else:
        entropy=0
    s_line_dur='Entropie = %f'%(entropy)
    screen_entropy_in.value=s_line_dur

In [22]:
im = interact(run_several_monobits,nb_tests=fixed(100),screen_result_in=fixed(screen_result),screen_entropy_in=fixed(screen_entropy))
im.widget.close()
im.widget.children[0].max=1
im.widget.children[0].min=0
im.widget.children[0].step=0.01
im.widget.children[0].description='Probabilite de generer un 1 :'
im.widget.children[0].style={'description_width': '30%'}
im.widget.children[0].layout=widgets.Layout(width='75%')

im.widget.children[1].max=4096
im.widget.children[1].min=0
im.widget.children[1].step=1
im.widget.children[1].description='Seuil Bas'
im.widget.children[1].style={'description_width': '30%'}
im.widget.children[1].layout=widgets.Layout(width='25%')

im.widget.children[2].max=4096
im.widget.children[2].min=0
im.widget.children[2].step=1
im.widget.children[2].description='Seuil Haut'
im.widget.children[2].style={'description_width': '30%'}
im.widget.children[2].layout=widgets.Layout(width='25%')

im.widget.children[3].max=4096
im.widget.children[3].min=0
im.widget.children[3].step=1
im.widget.children[3].description='Latence'
im.widget.children[3].style={'description_width': '30%'}
im.widget.children[3].layout=widgets.Layout(width='25%')

hb0=widgets.HBox((im.widget.children[0],screen_entropy))
hb1=widgets.HBox((im.widget.children[1],im.widget.children[2],im.widget.children[3],screen_result))
vb=widgets.VBox([hb0,hb1])
display(vb)

interactive(children=(FloatSlider(value=0.5, description='p', max=1.5, min=-0.5), IntSlider(value=512, descrip…